# 03. Basic Agent

## 개요
이 노트북은 **AutoGen 프레임워크를 사용한 기본 AI 에이전트** 구현 방법을 다룹니다.

### 주요 학습 내용
- Azure OpenAI와 AutoGen 연동
- 기본 에이전트 생성 및 도구(Tool) 활용
- 멀티모달 메시지 처리
- 스트리밍 응답 구현
- 메모리 기능을 가진 에이전트

## 1. 환경 설정 및 기본 날씨 에이전트

이 섹션에서는 **AutoGen 프레임워크의 기본 설정**과 **간단한 도구를 사용하는 에이전트**를 구현합니다.

### 주요 구성 요소
- **AzureOpenAIChatCompletionClient**: Azure OpenAI 모델과 연결
- **get_weather 도구**: 날씨 정보를 반환하는 사용자 정의 함수
- **AssistantAgent**: 도구를 사용할 수 있는 AI 에이전트
- **RoundRobinGroupChat**: 에이전트 팀 관리
- **대화형 루프**: 사용자 입력을 받아 실시간으로 응답

In [ ]:
import os

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from dotenv import load_dotenv

load_dotenv()

api_version = os.getenv("AZURE_OPENAI_API_VERSION")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
azure_openai_chat_completion_client = AzureOpenAIChatCompletionClient(
            model=deployment_name,
            azure_endpoint=azure_endpoint,
            api_version=api_version,
            api_key=api_key,
        )

# 도구 정의
async def get_weather(city: str) -> str:
    return f"{city}의 날씨는 23도이며 맑습니다."


async def main() -> None:
    # 에이전트 정의
    weather_agent = AssistantAgent(
        name="weather_agent",
        model_client=azure_openai_chat_completion_client,
        tools=[get_weather],
    )

    # 단일 에이전트로 구성된 팀을 정의하고 최대 자동 생성 턴을 1로 설정
    agent_team = RoundRobinGroupChat([weather_agent], max_turns=1)

    while True:
        # 콘솔에서 사용자 입력 받기
        user_input = input("도시 이름을 입력하세요. ('종료'를 입력하면 끝남): ")
        if user_input.strip().lower() in ["종료", "exit"]:
            break
        # 팀을 실행하고 콘솔에 메시지 스트리밍
        stream = agent_team.run_stream(task=user_input)
        await Console(stream)


# NOTE: if running this inside a Python script you'll need to use asyncio.run(main()).
await main()

: 

## 2. 멀티모달 메시지 처리

이 섹션에서는 **텍스트와 이미지를 함께 처리**할 수 있는 멀티모달 에이전트를 구현합니다.

### 주요 기능
- **TextMessage**: 간단한 텍스트 메시지 생성
- **MultiModalMessage**: 텍스트와 이미지를 조합한 메시지
- **PIL과 matplotlib**: 이미지 다운로드 및 표시
- **이미지 분석**: AI가 이미지 내용을 설명하는 기능

💡 **활용 사례**: 이미지 분석, 시각적 질문 답변, 문서 이해 등

In [ ]:
from autogen_agentchat.messages import TextMessage

text_message = TextMessage(content="안녕하세요!", source="User")

from io import BytesIO

import matplotlib.pyplot as plt
import requests
from autogen_agentchat.messages import MultiModalMessage
from autogen_core import CancellationToken
from autogen_core import Image as AGImage
from PIL import Image

pil_image = Image.open(BytesIO(requests.get("https://picsum.photos/300/200").content))
img = AGImage(pil_image)
multi_modal_message = MultiModalMessage(content=["이 이미지의 내용을 설명해주실 수 있나요?", img], source="user")
plt.imshow(pil_image) 
plt.axis('off') 
plt.show()

agent = AssistantAgent(
    name="assistant",
    model_client=azure_openai_chat_completion_client,
    system_message="작업을 해결하기 위해 도구를 사용하세요.",
)

async def assistant_run() -> None:
    response = await agent.on_messages(
        [multi_modal_message],
        cancellation_token=CancellationToken(),
    )

    print([response.chat_message.content])

    print(response.chat_message)



# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()

## 3. 웹 검색 도구를 가진 에이전트

이 섹션에서는 **사용자 정의 도구(웹 검색)**를 사용하는 에이전트를 구현합니다.

### 핵심 개념
- **Custom Tool**: `web_search` 함수로 사용자 정의 도구 생성
- **Tool Integration**: 에이전트가 도구를 자동으로 호출하는 방법
- **Structured Response**: 도구 호출 결과와 최종 응답을 구분해서 확인

🔍 **실습 목적**: AutoGen이 질문에 적합한 도구를 자동으로 선택하고 실행하는 과정을 이해

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken


# 웹에서 정보를 검색하는 도구 정의
async def web_search(query: str) -> str:
    """웹에서 정보 찾기"""
    return "AutoGen은 다중 에이전트 애플리케이션을 구축하기 위한 프로그래밍 프레임워크입니다."


# OpenAI GPT-4o 모델을 사용하는 에이전트 생성
model_client = azure_openai_chat_completion_client
agent = AssistantAgent(
    name="assistant",
    model_client=model_client,
    tools=[web_search],
    system_message="작업을 해결하기 위해 도구를 사용하세요.",
)

async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="AutoGen에 대한 정보를 찾아주세요", source="User")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message.content)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()

## 4. UserProxy 에이전트

이 섹션에서는 **사용자를 대변하는 UserProxy 에이전트**의 동작을 살펴봅니다.

### UserProxy 특징
- **사용자 역할 시뮬레이션**: 실제 사용자 대신 자동으로 응답
- **간단한 응답**: 복잡한 추론 없이 직접적인 답변 제공
- **Multi-Agent 시스템**: 다중 에이전트 시스템에서 사용자 인터페이스 역할

⚠️ **참고**: UserProxy는 보통 다른 에이전트와 협업할 때 더 유용합니다.

In [ ]:
from autogen_agentchat.agents import UserProxyAgent
from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken


async def user_proxy_run() -> None:
    user_proxy_agent = UserProxyAgent("user_proxy")
    response = await user_proxy_agent.on_messages(
        [TextMessage(content="당신의 이름은 무엇인가요? ", source="User")], cancellation_token=CancellationToken()
    )
    print(f"당신의 이름은 {response.chat_message.content}입니다")


# Use asyncio.run(user_proxy_run()) when running in a script.
await user_proxy_run()

## 5. 스트리밍 응답 구현

이 섹션에서는 **실시간 스트리밍 방식**으로 AI 응답을 받는 방법을 구현합니다.

### 스트리밍의 장점
- **실시간 피드백**: 응답이 생성되는 과정을 실시간으로 확인
- **향상된 UX**: 사용자가 기다리는 시간이 지루하지 않음
- **중간 결과 확인**: 도구 호출과 응답 생성 과정을 단계별로 관찰

### 두 가지 구현 방식
1. **Option 1**: `async for` 루프로 각 메시지를 개별 처리
2. **Option 2**: `Console` 클래스로 자동 UI 렌더링 (더 간편함)

In [ ]:
from autogen_agentchat.ui import Console


async def assistant_run_stream() -> None:
    # 옵션 1: 스트림에서 각 메시지를 읽기 (이전 예제에서 보여진 대로)
    # async for message in agent.on_messages_stream(
    #     [TextMessage(content="AutoGen에 대한 정보를 찾아주세요", source="user")],
    #     cancellation_token=CancellationToken(),
    # ):
    #     print(message)

    # 옵션 2: Console을 사용하여 나타나는 모든 메시지 출력
    await Console(
        agent.on_messages_stream(
            [TextMessage(content="AutoGen에 대한 정보를 찾아주세요", source="user")],
            cancellation_token=CancellationToken(),
        )
    )


# Use asyncio.run(assistant_run_stream()) when running in a script.
await assistant_run_stream()

## 6. 메모리 기능을 가진 에이전트

이 섹션에서는 **사용자 정보를 기억하는 메모리 기능**을 가진 에이전트를 구현합니다.

### 메모리 시스템 특징
- **ListMemory**: 메모리 내용을 리스트 형태로 저장
- **MemoryContent**: 텍스트 기반의 메모리 컨텐츠 관리
- **Persistent Context**: 이전 대화 내용과 사용자 선호도 기억
- **개인화**: 사용자별 맞춤형 응답 제공

### 활용 사례
- 사용자 프로필 관리
- 대화 컨텍스트 유지  
- 개인 비서 기능
- 추천 시스템

🧠 **핵심**: 에이전트가 단순한 질문-답변을 넘어서 **관계를 기억하고 발전**시킬 수 있는 능력을 제공합니다.

In [ ]:
from autogen_core.memory import ListMemory, MemoryContent, MemoryMimeType

# Initialize user memory
user_memory = ListMemory()

# 사용자 선호도를 메모리에 추가
await user_memory.add(MemoryContent(content="나는 고양이를 좋아해", mime_type=MemoryMimeType.TEXT))

await user_memory.add(MemoryContent(content="내 생일은 7월 31일이야", mime_type=MemoryMimeType.TEXT))

assistant_agent = AssistantAgent(
    name="assistant",
    model_client=azure_openai_chat_completion_client,
    memory=[user_memory],
)

stream = assistant_agent.run_stream(task="내 생일은 언제야? 내가 좋아하는 애완동물은?")
await Console(stream)

## 마무리

### 🎯 학습한 핵심 개념들

1. **기본 에이전트 구성**: Azure OpenAI + AutoGen 연동
2. **도구 통합**: 사용자 정의 함수를 에이전트가 자동으로 활용
3. **멀티모달 처리**: 텍스트와 이미지를 함께 다루는 AI 시스템
4. **스트리밍 응답**: 실시간 상호작용으로 사용자 경험 향상
5. **메모리 시스템**: 컨텍스트를 기억하는 개인화된 에이전트

### 🚀 다음 단계

- **Multi-Agent 시스템**: 여러 에이전트가 협업하는 복합 시스템
- **고급 도구 연동**: 실제 API, 데이터베이스, 외부 서비스 연결
- **워크플로 설계**: 복잡한 작업을 단계별로 처리하는 에이전트 파이프라인
- **성능 최적화**: 응답 속도와 정확도 향상을 위한 튜닝

**AutoGen으로 더 지능적이고 실용적인 AI 에이전트를 구축해보세요!** 🤖✨

---

# 종합 실습: 실제 API 기반 스마트 여행 계획 어시스턴트

이제 배운 모든 개념들을 활용해서 **실제 API와 연동된 production-ready 스마트 여행 계획 어시스턴트**를 만들어보겠습니다!

## 🆕 주요 업그레이드 사항
- ✅ **실제 API 통합**: OpenWeather, Google Places, Exchange Rate API 연동
- ✅ **한국어 시스템 메시지**: 콘텐츠 필터 친화적인 한국어 프롬프트
- ✅ **Fallback 시스템**: API 키 없어도 모의 데이터로 정상 동작
- ✅ **에러 처리**: 안정적인 API 호출 및 예외 처리

## 📋 구현된 고급 기능들
- 🌤️ **실시간 날씨 예보**: OpenWeather API로 실제 기상 정보 제공
- 🏛️ **실제 명소 검색**: Google Places API로 평점과 주소 포함 정보
- 💱 **실시간 환율 계산**: ExchangeRate API로 최신 환율 정보
- 🧠 **지능적 메모리**: 사용자 선호도 학습 및 개인화 서비스
- ⚡ **스트리밍 응답**: 실시간 대화형 인터페이스

## 🎯 학습 목표
이 실습을 통해 **실제 서비스에 바로 적용 가능한 AI 에이전트** 구축 능력을 습득할 수 있습니다.

### 🌟 특별한 점
- **실제 데이터**: 모의 데이터가 아닌 실시간 API 정보 활용
- **안전한 한국어 AI**: 콘텐츠 필터를 통과하는 전문적인 한국어 에이전트
- **확장 가능한 구조**: 추가 API 및 기능을 쉽게 통합할 수 있는 아키텍처

In [19]:
# ============================================
# 🌟 종합 실습: 스마트 여행 계획 어시스턴트
# ============================================

import asyncio
import json
import random
import aiohttp
import os
from datetime import datetime, timedelta
from typing import Dict, List, Optional

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken
from autogen_core.memory import ListMemory, MemoryContent, MemoryMimeType

# ============================
# 📡 도구 (Tools) 정의 
# ============================

async def get_weather_forecast(city: str, days: int = 3) -> str:
    """OpenWeatherMap API를 사용해서 날씨 예보를 조회합니다."""
    api_key = os.getenv("OPENWEATHER_API_KEY")
    
    if not api_key:
        # API 키가 없으면 모의 데이터 제공
        weather_conditions = ["맑음", "흐림", "비", "눈", "안개"]
        temperatures = list(range(15, 30))
        
        forecast = {"city": city, "forecast": []}
        
        for i in range(days):
            date = datetime.now() + timedelta(days=i)
            day_weather = {
                "date": date.strftime("%Y-%m-%d"),
                "condition": random.choice(weather_conditions),
                "temperature": f"{random.choice(temperatures)}°C",
                "humidity": f"{random.randint(40, 80)}%"
            }
            forecast["forecast"].append(day_weather)
        
        return json.dumps(forecast, ensure_ascii=False, indent=2)
    
    try:
        # 실제 OpenWeatherMap API 호출
        async with aiohttp.ClientSession() as session:
            # 1. 도시 좌표 얻기
            geo_url = f"http://api.openweathermap.org/geo/1.0/direct?q={city}&limit=1&appid={api_key}"
            async with session.get(geo_url) as response:
                if response.status != 200:
                    return f"죄송합니다. {city}의 위치 정보를 찾을 수 없습니다."
                
                geo_data = await response.json()
                if not geo_data:
                    return f"죄송합니다. {city}의 위치 정보를 찾을 수 없습니다."
                
                lat, lon = geo_data[0]['lat'], geo_data[0]['lon']
            
            # 2. 날씨 예보 가져오기
            weather_url = f"http://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid={api_key}&units=metric&lang=kr"
            async with session.get(weather_url) as response:
                if response.status != 200:
                    return f"죄송합니다. {city}의 날씨 정보를 가져올 수 없습니다."
                
                weather_data = await response.json()
                
                # 일별 예보 정리
                daily_forecasts = {}
                for item in weather_data['list'][:days*8]:  # 3시간 간격이므로 하루 8개
                    date = item['dt_txt'][:10]  # YYYY-MM-DD
                    if date not in daily_forecasts:
                        daily_forecasts[date] = {
                            "date": date,
                            "condition": item['weather'][0]['description'],
                            "temperature": f"{int(item['main']['temp'])}°C",
                            "humidity": f"{item['main']['humidity']}%"
                        }
                
                forecast = {
                    "city": city,
                    "forecast": list(daily_forecasts.values())[:days]
                }
                
                return json.dumps(forecast, ensure_ascii=False, indent=2)
                
    except Exception as e:
        return f"날씨 정보를 가져오는 중 오류가 발생했습니다: {str(e)}"

async def search_attractions(city: str, category: str = "관광지") -> str:
    """Google Places API를 사용해서 실제 명소를 검색합니다."""
    google_api_key = os.getenv("GOOGLE_PLACES_API_KEY")
    
    if not google_api_key:
        # API 키가 없으면 기본 데이터베이스 사용
        attractions_db = {
            "서울": {
                "관광지": ["경복궁", "남산타워", "명동", "홍대", "강남"],
                "맛집": ["광장시장", "명동교자", "신촌맛집거리", "이태원"],
                "숙박": ["롯데호텔", "신라호텔", "게스트하우스", "에어비앤비"]
            },
            "부산": {
                "관광지": ["해운대", "광안대교", "감천문화마을", "태종대"],
                "맛집": ["자갈치시장", "해운대맛집거리", "서면맛집"],
                "숙박": ["해운대호텔", "부산역근처숙소", "게스트하우스"]
            },
            "제주": {
                "관광지": ["한라산", "성산일출봉", "우도", "천지연폭포"],
                "맛집": ["흑돼지거리", "동문시장", "제주향토음식"],
                "숙박": ["제주신화월드", "펜션", "리조트"]
            }
        }
        
        city_data = attractions_db.get(city, {"관광지": ["정보를 찾을 수 없습니다"]})
        results = city_data.get(category, ["해당 카테고리 정보가 없습니다"])
        
        return f"{city}의 {category} 추천:\n" + "\n".join([f"- {item}" for item in results])
    
    try:
        # Google Places API를 사용한 실제 검색
        place_type_mapping = {
            "관광지": "tourist_attraction",
            "맛집": "restaurant", 
            "숙박": "lodging",
            "쇼핑": "shopping_mall",
            "카페": "cafe"
        }
        
        place_type = place_type_mapping.get(category, "tourist_attraction")
        
        async with aiohttp.ClientSession() as session:
            # Text Search API 사용
            search_url = "https://maps.googleapis.com/maps/api/place/textsearch/json"
            params = {
                "query": f"{city} {category}",
                "type": place_type,
                "key": google_api_key,
                "language": "ko"
            }
            
            async with session.get(search_url, params=params) as response:
                if response.status != 200:
                    return f"죄송합니다. {city}의 {category} 정보를 가져올 수 없습니다."
                
                data = await response.json()
                
                if not data.get("results"):
                    return f"{city}에서 {category} 정보를 찾을 수 없습니다."
                
                # 상위 10개 결과 추출
                places = []
                for place in data["results"][:10]:
                    name = place.get("name", "이름 없음")
                    rating = place.get("rating", 0)
                    address = place.get("formatted_address", "주소 정보 없음")
                    
                    place_info = f"{name}"
                    if rating > 0:
                        place_info += f" (평점: {rating})"
                    places.append(place_info)
                
                result = f"{city}의 {category} 추천 (Google Places API):\n"
                result += "\n".join([f"- {place}" for place in places])
                
                return result
                
    except Exception as e:
        return f"{category} 정보를 가져오는 중 오류가 발생했습니다: {str(e)}"

async def calculate_exchange_rate(from_currency: str, to_currency: str, amount: float) -> str:
    """실제 환율 API를 사용해서 환율을 계산합니다."""
    api_key = os.getenv("EXCHANGE_RATE_API_KEY")  # exchangerate-api.com API 키
    
    if not api_key:
        # API 키가 없으면 기본 환율 사용
        rates = {
            ("USD", "KRW"): 1320.5,
            ("JPY", "KRW"): 8.8,
            ("EUR", "KRW"): 1420.3,
            ("CNY", "KRW"): 181.2,
            ("KRW", "USD"): 0.000757,
            ("KRW", "JPY"): 0.1136,
            ("KRW", "EUR"): 0.0007,
            ("KRW", "CNY"): 0.0055
        }
        
        rate = rates.get((from_currency, to_currency))
        if not rate:
            return f"죄송합니다. {from_currency}에서 {to_currency}로의 환율 정보를 찾을 수 없습니다."
        
        converted_amount = amount * rate
        return f"{amount} {from_currency} = {converted_amount:,.2f} {to_currency} (예시 환율: {rate})"
    
    try:
        # 실제 환율 API 호출 (exchangerate-api.com 사용)
        async with aiohttp.ClientSession() as session:
            url = f"https://v6.exchangerate-api.com/v6/{api_key}/pair/{from_currency}/{to_currency}"
            
            async with session.get(url) as response:
                if response.status != 200:
                    return f"환율 정보를 가져올 수 없습니다. (HTTP {response.status})"
                
                data = await response.json()
                
                if data.get("result") != "success":
                    return f"환율 정보를 가져올 수 없습니다: {data.get('error-type', 'Unknown error')}"
                
                rate = data["conversion_rate"]
                converted_amount = amount * rate
                last_update = data.get("time_last_update_utc", "정보 없음")
                
                return f"""
💱 환율 변환 결과 (실시간 API):
{amount} {from_currency} = {converted_amount:,.2f} {to_currency}

📊 환율 정보:
- 환율: 1 {from_currency} = {rate} {to_currency}
- 마지막 업데이트: {last_update}
                """
                
    except Exception as e:
        return f"환율 계산 중 오류가 발생했습니다: {str(e)}"

async def estimate_travel_budget(destination: str, days: int, style: str = "중급") -> str:
    """여행 예산을 추정합니다."""
    budget_per_day = {
        "저예산": {"숙박": 50000, "식사": 40000, "교통": 20000, "관광": 30000},
        "중급": {"숙박": 120000, "식사": 80000, "교통": 40000, "관광": 60000},
        "고급": {"숙박": 250000, "식사": 150000, "교통": 80000, "관광": 120000}
    }
    
    daily_budget = budget_per_day.get(style, budget_per_day["중급"])
    total_budget = {}
    
    for category, daily_cost in daily_budget.items():
        total_budget[category] = daily_cost * days
    
    total_cost = sum(total_budget.values())
    
    budget_breakdown = f"""
💰 {destination} {days}일 여행 예산 ({style} 스타일)

📊 카테고리별 예산:
- 숙박: {total_budget['숙박']:,}원 ({daily_budget['숙박']:,}원/일)
- 식사: {total_budget['식사']:,}원 ({daily_budget['식사']:,}원/일)
- 교통: {total_budget['교통']:,}원 ({daily_budget['교통']:,}원/일)
- 관광: {total_budget['관광']:,}원 ({daily_budget['관광']:,}원/일)

💸 총 예상 비용: {total_cost:,}원
💡 팁: 실제 비용은 개인의 소비 패턴에 따라 달라질 수 있습니다.
    """
    
    return budget_breakdown

# ============================
# 🧠 스마트 여행 어시스턴트 생성
# ============================

# 사용자 메모리 초기화
user_travel_memory = ListMemory()

# 여행 어시스턴트 에이전트 생성 - 안전한 한국어 시스템 메시지
smart_travel_assistant = AssistantAgent(
    name="smart_travel_assistant",  
    model_client=azure_openai_chat_completion_client,
    tools=[get_weather_forecast, search_attractions, calculate_exchange_rate, estimate_travel_budget],
    memory=[user_travel_memory],
    system_message="""
당신은 전문 여행 계획 상담사입니다.

업무 범위:
1. 여행 목적지 정보 제공 및 제안
2. 실시간 날씨 예보 서비스 (OpenWeather API 활용)
3. 관광 명소, 식당, 숙박 시설 검색 (Google Places API 활용)
4. 여행 예산 계산 및 환율 변환 (실시간 환율 API 활용)
5. 개인 맞춤 여행 일정 수립

서비스 원칙:
- 정확하고 유용한 정보 제공
- 사용자 요구사항 정확히 파악
- 이전 대화 내용 참고 및 활용
- 실시간 데이터 우선 활용
- 안전한 여행을 위한 조언 제공

보유 기능:
- OpenWeather API를 통한 실시간 날씨 서비스
- Google Places API를 통한 실제 장소 정보 제공
- 실시간 환율 계산 시스템
- 사용자 선호도 저장 및 활용 기능
- 가족 친화적 여행 조언 서비스

전문적이고 신뢰할 수 있는 여행 상담 서비스를 제공하겠습니다. 사용자의 여행이 안전하고 유익한 경험이 되도록 최선을 다해 지원하겠습니다.
"""
)

### 🔑 실제 API 연동 설정 가이드

이 실습에서는 **실제 외부 API**를 사용하여 production-level의 AI 에이전트를 구축합니다.

#### 📋 통합된 API 서비스들

1. **OpenWeatherMap API** (실시간 날씨 정보)
   - 웹사이트: https://openweathermap.org/api
   - 환경변수: `OPENWEATHER_API_KEY`
   - 무료 플랜: 1,000회/일
   - **기능**: 실제 도시 좌표 기반 3일 날씨 예보

2. **Google Places API** (실제 장소 정보)
   - 웹사이트: https://developers.google.com/maps/documentation/places/web-service
   - 환경변수: `GOOGLE_PLACES_API_KEY`
   - 무료 크레딧: $200/월 
   - **기능**: 평점, 주소 포함 실제 관광지/맛집/숙박 정보

3. **Exchange Rate API** (실시간 환율)
   - 웹사이트: https://exchangerate-api.com
   - 환경변수: `EXCHANGE_RATE_API_KEY`
   - 무료 플랜: 1,500회/월
   - **기능**: 실시간 환율 및 마지막 업데이트 시간 정보

#### ⚙️ 환경변수 설정 방법

`.env` 파일에 다음과 같이 추가하세요:

```bash
# 기존 Azure 설정
AZURE_OPENAI_API_KEY=your_azure_key
AZURE_OPENAI_ENDPOINT=your_azure_endpoint
AZURE_OPENAI_DEPLOYMENT_NAME=your_deployment_name
AZURE_OPENAI_API_VERSION=2024-02-15-preview

# 새로 추가할 실제 API 키들
OPENWEATHER_API_KEY=your_openweather_api_key
GOOGLE_PLACES_API_KEY=your_google_places_api_key
EXCHANGE_RATE_API_KEY=your_exchange_rate_api_key
```

#### 🔧 스마트 Fallback 시스템

**API 키가 설정되지 않은 경우에도 모의 데이터로 정상 작동**하므로:
- ✅ 테스트 목적으로 바로 사용 가능
- ✅ 단계별 API 연동 학습 가능  
- ✅ 개발 환경에서 제약 없이 실습 가능

#### 🎓 학습 단계별 접근법
1. **1단계**: API 키 없이 모의 데이터로 기능 테스트
2. **2단계**: 하나씩 API 키를 추가하며 실제 데이터 확인
3. **3단계**: 모든 API 연동 후 production-ready 에이전트 완성

In [20]:
# ============================
# 📦 필요한 패키지 설치 및 API 테스트
# ============================

# aiohttp가 설치되지 않은 경우 설치
try:
    import aiohttp
    print("✅ aiohttp 패키지가 이미 설치되어 있습니다.")
except ImportError:
    print("📦 aiohttp 패키지를 설치합니다...")
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "aiohttp"])
    import aiohttp
    print("✅ aiohttp 패키지 설치 완료!")

# API 키 설정 상태 확인
import os
from dotenv import load_dotenv

load_dotenv()

api_keys = {
    "OpenWeather API": os.getenv("OPENWEATHER_API_KEY"),
    "Google Places API": os.getenv("GOOGLE_PLACES_API_KEY"), 
    "Exchange Rate API": os.getenv("EXCHANGE_RATE_API_KEY")
}

print("\n🔑 API 키 설정 상태:")
print("="*40)

for api_name, key in api_keys.items():
    if key:
        print(f"✅ {api_name}: 설정됨 ({key[:8]}...)")
    else:
        print(f"⚠️  {api_name}: 미설정 (모의 데이터 사용)")

print("\n💡 API 키가 미설정된 경우에도 모의 데이터로 정상 작동합니다!")

✅ aiohttp 패키지가 이미 설치되어 있습니다.

🔑 API 키 설정 상태:
✅ OpenWeather API: 설정됨 (1fb0489e...)
✅ Google Places API: 설정됨 (AIzaSyCu...)
✅ Exchange Rate API: 설정됨 (7e002d6d...)

💡 API 키가 미설정된 경우에도 모의 데이터로 정상 작동합니다!


### 📝 실전 API 기반 실습 예제들

아래 코드들을 하나씩 실행해보면서 **실제 API 연동 스마트 여행 어시스턴트**의 강력한 기능들을 체험해보세요!

#### 🚀 실습 하이라이트
- **실시간 데이터**: 실제 날씨, 장소, 환율 정보 활용
- **한국어 AI**: 콘텐츠 필터 통과 전문 한국어 에이전트  
- **지능적 기능**: 메모리 기반 개인화 서비스
- **Production Ready**: 실제 서비스에 바로 적용 가능한 수준

In [21]:
# ============================
# 🎮 실습 1: 사용자 선호도 저장
# ============================

async def setup_user_preferences():
    """사용자의 여행 선호도를 메모리에 저장합니다."""
    
    # 사용자 선호도 정보를 메모리에 추가
    preferences = [
        "나는 자연 경관을 좋아해요",
        "예산은 1일 30만원 정도로 생각하고 있어요", 
        "맛집 탐방을 매우 좋아해요",
        "사진 찍기를 좋아해서 인스타그램 명소를 선호해요",
        "혼자 여행하는 것을 선호해요"
    ]
    
    for pref in preferences:
        await user_travel_memory.add(
            MemoryContent(content=pref, mime_type=MemoryMimeType.TEXT)
        )
    
    print("✅ 사용자 선호도가 저장되었습니다!")
    print("저장된 선호도:")
    for i, pref in enumerate(preferences, 1):
        print(f"  {i}. {pref}")

# 선호도 설정 실행
await setup_user_preferences()

✅ 사용자 선호도가 저장되었습니다!
저장된 선호도:
  1. 나는 자연 경관을 좋아해요
  2. 예산은 1일 30만원 정도로 생각하고 있어요
  3. 맛집 탐방을 매우 좋아해요
  4. 사진 찍기를 좋아해서 인스타그램 명소를 선호해요
  5. 혼자 여행하는 것을 선호해요


In [22]:
# ============================
# 🎮 실습 2: 기본 여행 상담 (스트리밍)
# ============================

async def travel_consultation_streaming():
    """스트리밍 방식으로 여행 상담을 진행합니다."""
    
    user_question = "제주도 3박 4일 여행을 계획하고 있는데, 날씨는 어떨까요? 그리고 내 취향에 맞는 추천도 해주세요!"
    
    print("🤖 스마트 여행 어시스턴트가 답변하고 있습니다...\n")
    print("="*60)
    
    # 스트리밍 방식으로 응답 받기
    await Console(
        smart_travel_assistant.on_messages_stream(
            [TextMessage(content=user_question, source="user")],
            cancellation_token=CancellationToken(),
        )
    )

# 스트리밍 상담 실행
await travel_consultation_streaming()

🤖 스마트 여행 어시스턴트가 답변하고 있습니다...

---------- MemoryQueryEvent (smart_travel_assistant) ----------
[MemoryContent(content='나는 자연 경관을 좋아해요', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None), MemoryContent(content='예산은 1일 30만원 정도로 생각하고 있어요', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None), MemoryContent(content='맛집 탐방을 매우 좋아해요', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None), MemoryContent(content='사진 찍기를 좋아해서 인스타그램 명소를 선호해요', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None), MemoryContent(content='혼자 여행하는 것을 선호해요', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None)]
[MemoryContent(content='나는 자연 경관을 좋아해요', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None), MemoryContent(content='예산은 1일 30만원 정도로 생각하고 있어요', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None), MemoryContent(content='맛집 탐방을 매우 좋아해요', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None), MemoryContent(content='사진 찍기를 좋아해서 인스타그램 명소를 선호

In [23]:
# ============================
# 🎮 실습 3: 상세한 여행 계획 수립
# ============================

async def detailed_travel_planning():
    """상세한 여행 계획을 수립하는 시나리오입니다."""
    
    questions = [
        "부산 여행 2박 3일 예산을 중급 스타일로 계산해주세요",
        "100달러를 한국 돈으로 바꾸면 얼마인가요?", 
        "부산의 유명한 맛집들을 추천해주세요",
        "서울의 관광지도 같이 알려주세요"
    ]
    
    for i, question in enumerate(questions, 1):
        print(f"\n{'='*60}")
        print(f"📋 질문 {i}: {question}")
        print('='*60)
        
        response = await smart_travel_assistant.on_messages(
            [TextMessage(content=question, source="user")],
            cancellation_token=CancellationToken(),
        )
        
        print(f"🤖 응답: {response.chat_message.content}")
        
        # 잠시 대기 (실제 대화처럼 보이게)
        await asyncio.sleep(1)

# 상세 계획 실행
await detailed_travel_planning()


📋 질문 1: 부산 여행 2박 3일 예산을 중급 스타일로 계산해주세요
🤖 응답: 
💰 부산 3일 여행 예산 (중급 스타일)

📊 카테고리별 예산:
- 숙박: 360,000원 (120,000원/일)
- 식사: 240,000원 (80,000원/일)
- 교통: 120,000원 (40,000원/일)
- 관광: 180,000원 (60,000원/일)

💸 총 예상 비용: 900,000원
💡 팁: 실제 비용은 개인의 소비 패턴에 따라 달라질 수 있습니다.
    
🤖 응답: 
💰 부산 3일 여행 예산 (중급 스타일)

📊 카테고리별 예산:
- 숙박: 360,000원 (120,000원/일)
- 식사: 240,000원 (80,000원/일)
- 교통: 120,000원 (40,000원/일)
- 관광: 180,000원 (60,000원/일)

💸 총 예상 비용: 900,000원
💡 팁: 실제 비용은 개인의 소비 패턴에 따라 달라질 수 있습니다.
    

📋 질문 2: 100달러를 한국 돈으로 바꾸면 얼마인가요?

📋 질문 2: 100달러를 한국 돈으로 바꾸면 얼마인가요?
🤖 응답: 
💱 환율 변환 결과 (실시간 API):
100.0 USD = 142,827.99 KRW

📊 환율 정보:
- 환율: 1 USD = 1428.2799 KRW
- 마지막 업데이트: Sat, 01 Nov 2025 00:00:01 +0000
                
🤖 응답: 
💱 환율 변환 결과 (실시간 API):
100.0 USD = 142,827.99 KRW

📊 환율 정보:
- 환율: 1 USD = 1428.2799 KRW
- 마지막 업데이트: Sat, 01 Nov 2025 00:00:01 +0000
                

📋 질문 3: 부산의 유명한 맛집들을 추천해주세요

📋 질문 3: 부산의 유명한 맛집들을 추천해주세요
🤖 응답: 부산의 맛집 추천 (Google Places API):
- 연화덮밥 (평점: 4.9)
- 동녘돈까스 송도본점 (평점: 4.9)
- 술고당 (평점: 4.8)

In [24]:
# ============================
# 🎮 실습 4: 대화형 여행 계획 시뮬레이션
# ============================

async def interactive_travel_simulation():
    """실제 대화처럼 진행되는 여행 계획 시뮬레이션입니다."""
    
    conversation_flow = [
        {
            "user": "안녕하세요! 가족과 함께 갈 수 있는 여행지를 추천해주세요.",
            "context": "첫 인사 및 기본 요청"
        },
        {
            "user": "아이들이 즐겁게 놀 수 있는 곳으로 추천해주세요. 그리고 날씨도 중요해요.",
            "context": "구체적인 요구사항 추가"
        },
        {
            "user": "제주도가 좋을 것 같은데, 3일간 날씨 예보를 알려주세요.",
            "context": "구체적인 여행지 선택 및 날씨 문의"
        },
        {
            "user": "예산은 얼마나 필요할까요? 4명이서 2박 3일 중급 스타일로 계획하고 있습니다.",
            "context": "예산 계획 문의"
        },
        {
            "user": "마지막으로 아이들과 방문하기 좋은 관광지를 추천해주세요.",
            "context": "최종 관광지 추천 요청"
        }
    ]
    
    print("🎭 대화형 여행 계획 시뮬레이션 시작!")
    print("="*70)
    
    for i, turn in enumerate(conversation_flow, 1):
        print(f"\n👤 사용자 ({turn['context']})")
        print(f"💬 \"{turn['user']}\"\n")
        
        # 스트리밍으로 응답받기
        print("🤖 여행 어시스턴트:")
        await Console(
            smart_travel_assistant.on_messages_stream(
                [TextMessage(content=turn['user'], source="user")],
                cancellation_token=CancellationToken(),
            )
        )
        
        print("\n" + "-"*50)
        await asyncio.sleep(2)  # 자연스러운 대화 흐름을 위한 대기

# 대화형 시뮬레이션 실행
await interactive_travel_simulation()

🎭 대화형 여행 계획 시뮬레이션 시작!

👤 사용자 (첫 인사 및 기본 요청)
💬 "안녕하세요! 가족과 함께 갈 수 있는 여행지를 추천해주세요."

🤖 여행 어시스턴트:
---------- MemoryQueryEvent (smart_travel_assistant) ----------
[MemoryContent(content='나는 자연 경관을 좋아해요', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None), MemoryContent(content='예산은 1일 30만원 정도로 생각하고 있어요', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None), MemoryContent(content='맛집 탐방을 매우 좋아해요', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None), MemoryContent(content='사진 찍기를 좋아해서 인스타그램 명소를 선호해요', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None), MemoryContent(content='혼자 여행하는 것을 선호해요', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None)]
[MemoryContent(content='나는 자연 경관을 좋아해요', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None), MemoryContent(content='예산은 1일 30만원 정도로 생각하고 있어요', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None), MemoryContent(content='맛집 탐방을 매우 좋아해요', mime_type=<MemoryMimeType.TEXT: 'text/plain'>

### 🎯 고급 확장 아이디어

실제 API 기반 시스템을 더욱 발전시켜보고 싶다면 다음 기능들을 구현해보세요:

#### 🔧 API 기능 확장 아이디어

1. **추가 실제 API 통합**
   ```python
   # Booking.com API - 실제 숙박 예약 정보
   # Amadeus API - 항공편 정보 및 예약
   # Foursquare API - 상세 장소 리뷰 및 사진
   # Uber/Lyft API - 실시간 교통비 계산
   ```

2. **고급 데이터 처리**
   ```python
   # 실시간 여행 경보 정보 (외교부 API)
   # 코로나19 입국 규정 (각국 정부 API)
   # 실시간 항공료 변동 추적
   # 환율 변동 알림 시스템
   ```

3. **AI 기능 고도화**
   ```python
   # 여행 사진 AI 분석 (Azure Computer Vision API)
   # 다국어 번역 서비스 (Azure Translator API)
   # 음성 명령 처리 (Azure Speech API)
   # 개인화 추천 엔진 (Azure ML)
   ```

4. **Multi-Agent 협업 시스템**
   ```python
   # 전문가 에이전트 팀 구성:
   # - 날씨 전문가 (기상청 API 연동)
   # - 교통 전문가 (Google Maps API 연동)  
   # - 숙박 전문가 (Booking.com API 연동)
   # - 예산 전문가 (실시간 할인 정보 수집)
   ```

#### 🚀 실제 서비스화 로드맵

1. **백엔드 서비스화**
   ```python
   # FastAPI/Flask 기반 REST API 구축
   # Docker 컨테이너화
   # Azure App Service 또는 AWS EC2 배포
   # Redis 캐싱으로 API 호출 최적화
   ```

2. **프론트엔드 개발**
   ```python
   # React/Vue.js 웹 애플리케이션
   # Flutter/React Native 모바일 앱
   # 실시간 채팅 인터페이스 (WebSocket)
   # 지도 기반 시각화 (Google Maps API)
   ```

3. **데이터베이스 & 인프라**
   ```python
   # PostgreSQL/MongoDB 사용자 데이터 저장
   # Azure Cosmos DB 여행 기록 관리
   # Azure Application Insights 모니터링
   # CI/CD 파이프라인 구축 (GitHub Actions)
   ```

4. **비즈니스 로직 고도화**
   ```python
   # 실시간 가격 알림 시스템
   # 개인화 여행 패키지 추천
   # SNS 연동 여행 공유 기능
   # 리뷰 및 평점 시스템 통합
   ```

#### 💡 실제 적용 사례

이 실습 코드를 기반으로 다음과 같은 **실제 서비스**를 구축할 수 있습니다:

- 🏢 **여행사 고객 상담 챗봇**
- 📱 **개인 여행 계획 모바일 앱** 
- 🌐 **여행 커뮤니티 플랫폼의 AI 어시스턴트**
- 🏨 **호텔 컨시어지 서비스 자동화**
- ✈️ **항공사 고객 서비스 AI**

---

## 🏆 실제 API 기반 실습 완료!

### 🎉 축하합니다! 

이 고급 실습을 통해 다음과 같은 **실전 Production AI 에이전트 개발 능력**을 완전히 습득했습니다:

#### ✅ 마스터한 핵심 역량들

1. **? 실제 API 통합**: OpenWeather, Google Places, ExchangeRate API 완벽 연동
2. **🧠 지능적 시스템 설계**: 메모리, Fallback, 에러 처리가 포함된 안정적 아키텍처
3. **🌏 다국어 AI 구현**: 콘텐츠 필터 친화적 한국어 전문 에이전트 개발
4. **⚡ 고급 UX 패턴**: 스트리밍, 실시간 데이터, 개인화 서비스
5. **🔒 Production 수준 안정성**: API 키 관리, 에러 처리, 확장 가능한 설계

#### 🚀 다음 레벨 도전 과제들

- **🏗️ Microservices 아키텍처**: 각 API를 독립적 서비스로 분리
- **📊 실시간 분석 시스템**: 사용자 패턴 분석 및 개인화 알고리즘
- **🤖 Multi-Agent Orchestration**: 전문가 에이전트들의 협업 워크플로  
- **🌍 글로벌 서비스**: 다국가 API 통합 및 지역별 맞춤화
- **💼 Enterprise 솔루션**: B2B 여행 서비스 플랫폼 구축

#### 🎖️ 획득한 전문 스킬셋

> **"API 기반 Production AI 에이전트 개발자"**
> 
> ✨ 실제 외부 서비스와 연동된 AI 시스템 구축 능력  
> ✨ 안정적이고 확장 가능한 에이전트 아키텍처 설계 능력  
> ✨ 다국어 및 멀티모달 AI 시스템 개발 능력  
> ✨ Production 환경에서의 에러 처리 및 모니터링 능력  

### 💡 전문가 팁

> **"Real-World Integration First, Optimization Second"**
> 
> 실제 API와 외부 시스템 연동 경험이 AI 에이전트 개발의 핵심입니다.  
> 이론보다는 실제 데이터와 서비스를 다루는 능력이 차별화의 포인트입니다!